# Chapter 9: The Iteration Protocol

**Chapter 9 - Learning Python, 5th Edition**

Python's iteration protocol is the engine behind `for` loops, comprehensions,
unpacking, and many built-in functions. Understanding the distinction between
**iterables** and **iterators** -- and the `__iter__` / `__next__` dunder
methods that power them -- is essential to writing Pythonic code.

## Key Concepts
- **Iterable**: An object with an `__iter__` method that returns an iterator
- **Iterator**: An object with a `__next__` method that produces successive values
- **StopIteration**: The signal that iteration is complete
- **`iter()` with sentinel**: Two-argument form for reading until a value
- **`reversed()`**: Reverse iteration via `__reversed__`

## Section 1: Iterable vs Iterator

An **iterable** is any object that can return an iterator (has `__iter__`).
An **iterator** is a stateful object that remembers where it is during
iteration and produces the next value via `__next__`.

Lists, tuples, strings, dicts, and sets are all iterables -- but they are
*not* iterators themselves. Calling `iter()` on them produces an iterator.

In [ ]:
# A list is iterable but is NOT an iterator
numbers: list[int] = [10, 20, 30]

print(f"numbers has __iter__: {hasattr(numbers, '__iter__')}")
print(f"numbers has __next__: {hasattr(numbers, '__next__')}")

# Calling iter() on the list produces an iterator
it = iter(numbers)
print(f"\ntype(it): {type(it)}")
print(f"it has __iter__: {hasattr(it, '__iter__')}")
print(f"it has __next__: {hasattr(it, '__next__')}")

# The iterator yields values one at a time
print(f"\nnext(it): {next(it)}")
print(f"next(it): {next(it)}")
print(f"next(it): {next(it)}")

# The iterable itself is unchanged -- you can create new iterators
it2 = iter(numbers)
print(f"\nFresh iterator: next(it2) = {next(it2)}")

# An iterator's __iter__ returns itself (iterators are also iterable)
print(f"\niter(it2) is it2: {iter(it2) is it2}")

## Section 2: StopIteration and Exhaustion

When an iterator has no more values, calling `next()` raises `StopIteration`.
The `for` loop catches this automatically. Once exhausted, an iterator cannot
be restarted -- you must create a new one from the iterable.

In [ ]:
colors: list[str] = ["red", "green"]
it = iter(colors)

print(f"next(it): {next(it)}")
print(f"next(it): {next(it)}")

# Iterator is now exhausted
try:
    next(it)
except StopIteration:
    print("\nStopIteration raised -- iterator is exhausted")

# Exhausted iterators produce nothing in for loops
remaining = list(it)
print(f"Remaining after exhaustion: {remaining}")

# You can use next() with a default to avoid the exception
it2 = iter(colors)
print(f"\nnext with default: {next(it2, 'DONE')}")
print(f"next with default: {next(it2, 'DONE')}")
print(f"next with default: {next(it2, 'DONE')}")

## Section 3: How `for` Works Under the Hood

The `for` loop is syntactic sugar for the iteration protocol. Internally,
Python calls `iter()` on the iterable, then repeatedly calls `next()` on
the resulting iterator until `StopIteration` is raised.

In [ ]:
# What a for loop ACTUALLY does (desugared)
data: list[str] = ["alpha", "beta", "gamma"]

print("=== Normal for loop ===")
for item in data:
    print(f"  {item}")

print("\n=== Desugared equivalent ===")
_iter = iter(data)         # Step 1: get an iterator
while True:
    try:
        item = next(_iter)  # Step 2: get next value
    except StopIteration:
        break               # Step 3: stop when exhausted
    print(f"  {item}")      # Step 4: execute loop body

# This also explains why iterators work directly in for loops:
# iter() on an iterator returns itself, so the protocol still works
print("\n=== Iterator directly in for loop ===")
it = iter(data)
next(it)  # Skip 'alpha'
for item in it:
    print(f"  {item} (started mid-stream)")

## Section 4: Building a Custom Iterator Class

To make a class iterable, implement `__iter__` (returns an iterator) and
`__next__` (produces values, raises `StopIteration` when done). If the
iterable *is* its own iterator, `__iter__` returns `self`.

A cleaner pattern separates the **iterable** (the container) from the
**iterator** (the cursor), so the same object can be iterated multiple times.

In [ ]:
from typing import Iterator


class FibonacciIterator:
    """Iterator that produces Fibonacci numbers up to a limit."""

    def __init__(self, max_value: int) -> None:
        self._max = max_value
        self._a: int = 0
        self._b: int = 1

    def __iter__(self) -> "FibonacciIterator":
        return self

    def __next__(self) -> int:
        if self._a > self._max:
            raise StopIteration
        value = self._a
        self._a, self._b = self._b, self._a + self._b
        return value


# Use in a for loop
print("Fibonacci numbers up to 100:")
for n in FibonacciIterator(100):
    print(f"  {n}")

# Works with list(), sum(), and other consumers
fibs = list(FibonacciIterator(50))
print(f"\nAs list: {fibs}")
print(f"Sum: {sum(FibonacciIterator(50))}")

# Caveat: this is a self-iterator, so it can only be iterated once
fib = FibonacciIterator(10)
print(f"\nFirst pass: {list(fib)}")
print(f"Second pass: {list(fib)}  (exhausted)")

In [ ]:
from typing import Iterator


class Range:
    """A re-iterable range-like object that separates iterable from iterator."""

    def __init__(self, start: int, stop: int, step: int = 1) -> None:
        self.start = start
        self.stop = stop
        self.step = step

    def __iter__(self) -> Iterator[int]:
        """Return a fresh iterator each time -- allows multiple iterations."""
        current = self.start
        while current < self.stop:
            yield current
            current += self.step

    def __len__(self) -> int:
        return max(0, (self.stop - self.start + self.step - 1) // self.step)

    def __repr__(self) -> str:
        return f"Range({self.start}, {self.stop}, {self.step})"


r = Range(1, 6)
print(f"r = {r}")
print(f"len(r) = {len(r)}")

# Can be iterated multiple times (unlike the FibonacciIterator)
print(f"\nFirst pass:  {list(r)}")
print(f"Second pass: {list(r)}")

# Works with all iteration consumers
print(f"\nsum(Range(1, 11)):    {sum(Range(1, 11))}")
print(f"max(Range(5, 20, 3)): {max(Range(5, 20, 3))}")
print(f"sorted(Range(10, 0, -1)): won't work with step < 0 (our simple impl)")

# With step
print(f"\nRange(0, 10, 2): {list(Range(0, 10, 2))}")
print(f"Range(0, 10, 3): {list(Range(0, 10, 3))}")

## Section 5: `iter()` with a Sentinel Value

The two-argument form `iter(callable, sentinel)` calls the callable
repeatedly until it returns the sentinel value, then raises `StopIteration`.
This is useful for reading from streams or sockets until a terminator.

In [ ]:
import random

# Simulate reading from a data source until we see a sentinel
random.seed(42)
data_source: list[str] = ["data1", "data2", "data3", "END", "data4"]
index = -1


def read_next() -> str:
    """Simulate reading from a stream."""
    global index
    index += 1
    return data_source[index]


# iter(callable, sentinel) -- stops when callable returns "END"
print("Reading until sentinel 'END':")
for value in iter(read_next, "END"):
    print(f"  Received: {value}")

print()

# Practical example: reading fixed-size blocks from a file-like object
import io

content = "Hello World! This is a test of block reading."
stream = io.StringIO(content)
BLOCK_SIZE = 10

# Read 10-char blocks until we get an empty string (EOF)
print(f"Reading in {BLOCK_SIZE}-char blocks:")
blocks: list[str] = []
for block in iter(lambda: stream.read(BLOCK_SIZE), ""):
    blocks.append(block)
    print(f"  Block: {block!r}")

print(f"\nReconstructed: {''.join(blocks)!r}")

## Section 6: `reversed()` and `__reversed__`

`reversed()` returns a reverse iterator. For sequences (list, tuple, range)
it works automatically. For custom classes, implement `__reversed__` or
fall back to `__len__` + `__getitem__`.

In [ ]:
from typing import Iterator

# Built-in sequences support reversed() out of the box
letters: list[str] = ["a", "b", "c", "d"]
print(f"reversed list: {list(reversed(letters))}")
print(f"reversed range: {list(reversed(range(1, 6)))}")
print(f"reversed string: {''.join(reversed('Python'))}")


# Custom class with __reversed__
class Countdown:
    """An iterable that counts down, with explicit reverse support."""

    def __init__(self, start: int) -> None:
        self.start = start

    def __iter__(self) -> Iterator[int]:
        """Count down from start to 1."""
        current = self.start
        while current >= 1:
            yield current
            current -= 1

    def __reversed__(self) -> Iterator[int]:
        """Count UP from 1 to start (reverse of counting down)."""
        current = 1
        while current <= self.start:
            yield current
            current += 1


cd = Countdown(5)
print(f"\nCountdown:          {list(cd)}")
print(f"Reversed countdown: {list(reversed(cd))}")

# Without __reversed__, Python falls back to __len__ + __getitem__
class Deck:
    """A simple indexed collection -- reversed() uses __len__ + __getitem__."""

    def __init__(self, cards: list[str]) -> None:
        self._cards = cards

    def __len__(self) -> int:
        return len(self._cards)

    def __getitem__(self, index: int) -> str:
        return self._cards[index]


deck = Deck(["Ace", "King", "Queen", "Jack"])
print(f"\nDeck forward:  {list(deck)}")
print(f"Deck reversed: {list(reversed(deck))}")

## Section 7: Iteration Across Built-in Types

Many Python types support the iteration protocol in different ways.
Understanding what you get when you iterate over each type is important.

In [ ]:
# Iterating over different built-in types

# Strings iterate over characters
print("String iteration:")
for ch in "Hi!":
    print(f"  {ch!r}")

# Dicts iterate over keys by default
config: dict[str, int] = {"timeout": 30, "retries": 3, "port": 8080}
print("\nDict iteration (keys):")
for key in config:
    print(f"  {key} -> {config[key]}")

# Use .items() for key-value pairs
print("\nDict .items():")
for key, value in config.items():
    print(f"  {key} = {value}")

# Sets iterate in arbitrary order
unique: set[int] = {3, 1, 4, 1, 5, 9}
print(f"\nSet iteration: {[x for x in unique]}")

# Files iterate over lines
import io
fake_file = io.StringIO("line 1\nline 2\nline 3\n")
print("\nFile iteration:")
for line in fake_file:
    print(f"  {line.rstrip()!r}")

# enumerate(), zip(), map() all return iterators
print(f"\ntype(enumerate([])): {type(enumerate([])).__name__}")
print(f"type(zip([], [])):    {type(zip([], [])).__name__}")
print(f"type(map(str, [])):   {type(map(str, [])).__name__}")

## Summary

### The Iteration Protocol
1. **Iterable**: Has `__iter__()` that returns an iterator
2. **Iterator**: Has `__next__()` that returns values, raises `StopIteration` when done
3. All iterators are iterable (`__iter__` returns `self`), but not vice versa

### `for` Loop Desugared
```python
_it = iter(iterable)      # calls __iter__
while True:
    try:
        value = next(_it)  # calls __next__
    except StopIteration:
        break
    # loop body with value
```

### Key Patterns
- Separate iterable from iterator for re-iterability (`__iter__` returns a new iterator via `yield`)
- Use `iter(callable, sentinel)` for stream-style reading
- Implement `__reversed__` for custom reverse iteration
- Use `next(it, default)` to avoid `StopIteration` exceptions